In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
sys.path.append(os.path.realpath('..'))

In [17]:
import numpy as np
import pandas as pd
import plotly.express as ex
from copy import deepcopy

from tracking_v2.target import SingleTurnTarget
from tracking_v2.kalman import LinearKalmanFilter
from tracking_v2.motion import ConstantVelocityModel
from tracking_v2.sensor import GeometricSensor

In [23]:
def run_one(n, target, sensor, kf):
    T = 1
    t = 0
    
    positions = target.true_states(T, n+1)[:, :3]
    
    m = sensor.generate_measurement(t, positions[0, :])
    kf.initialize(m.z, m.R)

    x_hat, P_hat = [], []
    for position in positions[1:, :]:
        t += T
        
        kf.predict(T)

        x_hat.append(kf.x_hat)
        P_hat.append(kf.P_hat)

        m = sensor.generate_measurement(t, position)
        kf.update(m.z, m.R)

    x_hat, P_hat = np.array(x_hat), np.array(P_hat)
    
    assert x_hat.shape[0] == n
    assert P_hat.shape[0] == n
    
    return x_hat, P_hat

def run_mc(m, n, target, sensor, kf, seeds=None):
    if seeds is None:
        seeds = np.arange(m)
    
    x_hat, P_hat = [], []
    for seed in seeds:
        kf = deepcopy(kf)
        x, P = run_one(n, target, sensor, kf)
        x_hat.append(x)
        P_hat.append(P)
    
    x_hat, P_hat = np.array(x_hat), np.array(P_hat)

    assert x_hat.shape[0] == m
    assert P_hat.shape[0] == m
    
    return x_hat, P_hat

In [24]:
target = SingleTurnTarget()
sensor = GeometricSensor()

kf = LinearKalmanFilter(ConstantVelocityModel(), [[1, 0, 0, 0, 0, 0],
                                                  [0, 1, 0, 0, 0, 0],
                                                  [0, 0, 1, 0, 0, 0]])

x_hat, P_hat = run_mc(100, 400, target, sensor, kf)

In [25]:
x_hat.shape

(100, 400, 6, 1)